In [1]:
import tkinter as tk
import heapq
from collections import deque
import time


In [2]:
visited_nodes = 0
final_cost = 0
execution_time = 0
final_path = []

## **Maze**


In [3]:
# Maze Matrix
maze = [
    [1, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 0, 1, 1, 1],
    [0, 1, 0, 1, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 0, 1]
]

start = (0, 0)
goal  = (7, 7)

ROWS, COLS = len(maze), len(maze[0])
CELL = 50

In [4]:
# Helpers
def neighbors(node):
    r, c = node
    for dr, dc in [(1,0),(-1,0),(0,1),(0,-1)]:
        nr, nc = r+dr, c+dc
        if 0 <= nr < ROWS and 0 <= nc < COLS and maze[nr][nc] == 1:
            yield (nr, nc)

## A* Algorithm**

In [5]:
def heuristic(a, b):
    # Manhattan distance
    return abs(a[0]-b[0]) + abs(a[1]-b[1])
def astar():
    pq = []
    heapq.heappush(pq, (0, start))
    came_from = {}
    g_cost = {start: 0}
    visited = set()
    visited_list = []

    while pq:
        _, current = heapq.heappop(pq)
        if current == goal:
            break
        if current in visited:
            continue
        visited.add(current)
        visited_list.append(current)

        for neighbor in neighbors(current):
            if neighbor in visited:
                continue
            new_g = g_cost[current] + 1
            if neighbor not in g_cost or new_g < g_cost[neighbor]:
                g_cost[neighbor] = new_g
                f = new_g + heuristic(neighbor, goal)
                heapq.heappush(pq, (f, neighbor))
                came_from[neighbor] = current

    path = []
    node = goal
    while node != start:
        path.append(node)
        node = came_from[node]
    path.append(start)
    path.reverse()
    return path, visited_list

## **DLS**

In [6]:
def dls(start, goal, limit):
    path = []
    visited = set()
    visited_list = []

    def dfs_recursive(node, depth):
        if depth < 0:
            return False
        visited.add(node)
        visited_list.append(node)
        path.append(node)
        if node == goal:
            return True
        for nxt in neighbors(node):
            if nxt not in visited:
                if dfs_recursive(nxt, depth-1):
                    return True
        path.pop()
        return False

    dfs_recursive(start, limit)
    return path, visited_list

## **UCS**

In [7]:
def ucs(maze, start, goal):
    pq = []
    heapq.heappush(pq, (0, start, [start]))
    visited = set()
    visited_list = []

    while pq:
        cost, node, path = heapq.heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        visited_list.append(node)
        if node == goal:
            return path, visited_list

        for nxt in neighbors(node):
            if nxt not in visited:
                heapq.heappush(pq, (cost+1, nxt, path + [nxt]))
    return [], visited_list


## **BFS**

In [8]:
def bfs_maze(maze, start, goal):
    q = deque([start])
    parent = {start: None}
    visited = set([start])
    visited_list = [start]

    while q:
        r, c = q.popleft()
        if (r,c) == goal:
            break

        for nr, nc in neighbors((r,c)):
            if (nr,nc) not in visited:
                visited.add((nr,nc))
                visited_list.append((nr,nc))
                parent[(nr,nc)] = (r,c)
                q.append((nr,nc))

    path = []
    node = goal
    while node is not None:
        path.append(node)
        node = parent.get(node)
    path.reverse()
    return path, visited_list

## **Greedy**

In [9]:
def heuristic(a, b):
    # Manhattan distance
    return abs(a[0]-b[0]) + abs(a[1]-b[1])
# ====== Greedy Best-First Search ======
def solve_greedy(maze, start, end):
    dirs = [(-1,0),(0,1),(1,0),(0,-1)]
    def h(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])
    rows,cols = len(maze),len(maze[0])
    heap = [(h(start,end),start)]
    parent = {start:None}
    visited = set([start])
    visited_list = [start]  

    while heap:
        _,curr = heapq.heappop(heap)
        if curr == end:
            break
        for dr,dc in dirs:
            nr,nc = curr[0]+dr, curr[1]+dc
            if 0<=nr<rows and 0<=nc<cols and maze[nr][nc]==1:
                nxt = (nr,nc)
                if nxt not in visited:
                    visited.add(nxt)
                    visited_list.append(nxt)
                    parent[nxt] = curr
                    heapq.heappush(heap,(h(nxt,end),nxt))

    if end not in parent:
        return [], visited_list

    path = []
    cur = end
    while cur is not None:
        path.append(cur)
        cur = parent[cur]
    path.reverse()
    return path, visited_list



## **DFS**

In [10]:
# Depth-First Search 
def dfs(maze, start, goal):
    stack = [(start, [start])]
    visited = set()
    visited_list = []  
    rows, cols = len(maze), len(maze[0])

    while stack:
        (x, y), path = stack.pop()
        if (x, y) == goal:
            return path, visited_list
        if (x, y) in visited:
            continue
        visited.add((x, y))
        visited_list.append((x, y))
        for dx, dy in [(0,1),(0,-1),(1,0),(-1,0)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 1:
                stack.append(((nx, ny), path+[(nx, ny)]))
    return [], visited_list


## **GUI**

In [11]:
CELL = 50

ROWS = len(maze)
COLS = len(maze[0])

# ================== GUI ==================
root = tk.Tk()
root.title("Search Algorithms GUI")

canvas = tk.Canvas(root, width=COLS*CELL, height=ROWS*CELL)
canvas.pack()

algo = tk.StringVar(value="A*")

info_label = tk.Label(
    root,
    text="Cost: 0 | Visited: 0 | Time: 0.000s",
    font=("Arial", 11)
)
info_label.pack(pady=5)

# ----- DRAW -----
def draw(path=None, visited=None):
    canvas.delete("all")
    visited = visited or set()
    for r in range(ROWS):
        for c in range(COLS):
            x1, y1 = c*CELL, r*CELL
            x2, y2 = x1+CELL, y1+CELL
            color = "black" if maze[r][c]==0 else "white"
            if (r,c) in visited:
                color = "#87CEFA"
            if path and (r,c) in path:
                color = "pink"
            if (r,c)==start:
                color = "blue"
            elif (r,c)==goal:
                color = "green"
            canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="gray")

# ----- ANIMATE -----
def animate_visited(visited_list, path, exec_time):
    visited_set = set()
    for node in visited_list:
        visited_set.add(node)
        draw(path=None, visited=visited_set)
        info_label.config(text=f"Cost: -- | Visited: {len(visited_set)} | Time: {exec_time:.4f}s")
        root.update()
        root.after(20)
    draw(path=path, visited=visited_set)
    info_label.config(text=f"Cost: {len(path)-1} | Visited: {len(visited_set)} | Time: {exec_time:.4f}s")
    

# ----- RUN -----
def run():
    import time

    start_time = time.time()

    if algo.get() == "A*":
        path, visited_list = astar()
    elif algo.get() == "BFS":
        path, visited_list = bfs_maze(maze, start, goal)
    elif algo.get() == "DFS":
        path, visited_list = dfs(maze, start, goal)
    elif algo.get() == "Greedy":
        path, visited_list = solve_greedy(maze, start, goal)
    elif algo.get() == "DLS":
        path, visited_list = dls(start, goal, 40)
    elif algo.get() == "UCS":
        path, visited_list = ucs(maze, start, goal)
    else:
        return

    exec_time = time.time() - start_time  # ⏱️ وقت الخوارزمية فقط

    animate_visited(visited_list, path, exec_time)
    path_label.config(text="Path: " + " -> ".join(str(p) for p in path))


# ---------- MENU ----------
option_menu = tk.OptionMenu(
    root, algo,
    "A*", "BFS", "DFS", "Greedy", "DLS", "UCS"  # Removed IDDFS & Bidirectional
)
option_menu.config(bg="#FF69B4", fg="white", font=("Arial",11),
                   activebackground="#FF1493", activeforeground="white")
option_menu["menu"].config(bg="white", fg="black", font=("Arial",10))
option_menu.pack(pady=5)

run_btn = tk.Button(
    root,
    text="Run Search",
    font=("Arial",12,"bold"),
    bg="#FF69B4", fg="white",
    activebackground="#FF1493", activeforeground="white",
    padx=10, pady=5,
    command=run
)

run_btn.pack(pady=10)
path_label = tk.Label(
    root,
    text="Path: ",
    font=("Arial", 10),
    wraplength=COLS*CELL  
)
path_label.pack(pady=5)

draw()
root.mainloop()
